In [1]:
import os
import random
import tensorflow as tf
import time
import numpy as np
import cv2
from datetime import timedelta
slim = tf.contrib.slim

In [4]:
checkpoint_dir = 'checkpoints/'

In [5]:
def restore_session():
    global_step = 0
    if not os.path.exists(checkpoint_dir):
        raise IOError(checkpoint_dir + ' does not exist.')
    else:
        path = tf.train.get_checkpoint_state(checkpoint_dir)
        if path is None:
            raise IOError('No checkpoint to restore in ' + self.checkpoint_dir)
        else:
            saver.restore(session, path.model_checkpoint_path)
            global_step = int(path.model_checkpoint_path.split('-')[-1])

    return global_step

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
    
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [14]:
def conv_layer(inputs, W_shape, b_shape, name, padding='SAME'):
    W = weight_variable(W_shape)
    b = bias_variable([b_shape])
    layer = tf.nn.conv2d(inputs, W, strides=[1, 1, 1, 1], padding=padding) + b
    layer = slim.batch_norm(layer)
    layer = tf.nn.relu(layer)
    return layer       

In [8]:
def pool_layer(inputs, padding='SAME'):
    layer, pool_argmax = tf.nn.max_pool_with_argmax(inputs, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding=padding)
    return layer, pool_argmax

In [18]:
def deconv_layer(inputs, W_shape, b_shape, name, padding='SAME'):
    W = weight_variable(W_shape)
    b = bias_variable([b_shape])

    x_shape = tf.shape(inputs)
    out_shape = tf.pack([x_shape[0], x_shape[1], x_shape[2], W_shape[2]])

    layer = tf.nn.conv2d_transpose(inputs, W, out_shape, [1, 1, 1, 1], padding=padding) + b
    layer = slim.batch_norm(layer)
    layer = tf.nn.relu(layer)
    
    return layer

In [10]:
def unravel_argmax(argmax, shape):
    output_list = []
    output_list.append(argmax // (shape[2] * shape[3]))
    output_list.append(argmax % (shape[2] * shape[3]) // shape[3])
    return tf.pack(output_list)

In [11]:
def unpool_layer2x2(inputs, raveled_argmax, out_shape):
    argmax = unravel_argmax(raveled_argmax, tf.to_int64(out_shape))
    output = tf.zeros([out_shape[1], out_shape[2], out_shape[3]])

    height = tf.shape(output)[0]
    width = tf.shape(output)[1]
    channels = tf.shape(output)[2]

    t1 = tf.to_int64(tf.range(channels))
    t1 = tf.tile(t1, [((width + 1) // 2) * ((height + 1) // 2)])
    t1 = tf.reshape(t1, [-1, channels])
    t1 = tf.transpose(t1, perm=[1, 0])
    t1 = tf.reshape(t1, [channels, (height + 1) // 2, (width + 1) // 2, 1])

    t2 = tf.squeeze(argmax)
    t2 = tf.pack((t2[0], t2[1]), axis=0)
    t2 = tf.transpose(t2, perm=[3, 1, 2, 0])

    t = tf.concat(3, [t2, t1])
    indices = tf.reshape(t, [((height + 1) // 2) * ((width + 1) // 2) * channels, 3])

    x1 = tf.squeeze(inputs)
    x1 = tf.reshape(x1, [-1, channels])
    x1 = tf.transpose(x1, perm=[1, 0])
    values = tf.reshape(x1, [-1])

    delta = tf.SparseTensor(indices, values, tf.to_int64(tf.shape(output)))
    return tf.expand_dims(tf.sparse_tensor_to_dense(tf.sparse_reorder(delta)), 0)

In [20]:
session = tf.Session()
x = tf.placeholder(tf.float32, shape=(1, None, None, 3))
y_true = tf.placeholder(tf.int64, shape=(1, None, None))
expected = tf.expand_dims(y_true, -1)
rate = tf.placeholder(tf.float32, shape=[])

conv_1_1 = conv_layer(inputs=x, W_shape=[3, 3, 3, 64], b_shape=64, name='conv_1_1')
conv_1_2 = conv_layer(conv_1_1, [3, 3, 64, 64], 64, 'conv_1_2')

pool_1, pool_1_argmax = pool_layer(conv_1_2)

conv_2_1 = conv_layer(pool_1, [3, 3, 64, 128], 128, 'conv_2_1')
conv_2_2 = conv_layer(conv_2_1, [3, 3, 128, 128], 128, 'conv_2_2')

pool_2, pool_2_argmax = pool_layer(conv_2_2)

conv_3_1 = conv_layer(pool_2, [3, 3, 128, 256], 256, 'conv_3_1')
conv_3_2 = conv_layer(conv_3_1, [3, 3, 256, 256], 256, 'conv_3_2')
conv_3_3 = conv_layer(conv_3_2, [3, 3, 256, 256], 256, 'conv_3_3')

pool_3, pool_3_argmax = pool_layer(conv_3_3)

conv_4_1 = conv_layer(pool_3, [3, 3, 256, 512], 512, 'conv_4_1')
conv_4_2 = conv_layer(conv_4_1, [3, 3, 512, 512], 512, 'conv_4_2')
conv_4_3 = conv_layer(conv_4_2, [3, 3, 512, 512], 512, 'conv_4_3')

pool_4, pool_4_argmax = pool_layer(conv_4_3)

conv_5_1 = conv_layer(pool_4, [3, 3, 512, 512], 512, 'conv_5_1')
conv_5_2 = conv_layer(conv_5_1, [3, 3, 512, 512], 512, 'conv_5_2')
conv_5_3 = conv_layer(conv_5_2, [3, 3, 512, 512], 512, 'conv_5_3')

pool_5, pool_5_argmax = pool_layer(conv_5_3)

fc_6 = conv_layer(pool_5, [7, 7, 512, 4096], 4096, 'fc_6')
fc_7 = conv_layer(fc_6, [1, 1, 4096, 4096], 4096, 'fc_7')

deconv_fc_6 = deconv_layer(fc_7, [7, 7, 512, 4096], 512, 'fc6_deconv', padding='VALID')

unpool_5 = unpool_layer2x2(deconv_fc_6, pool_5_argmax, tf.shape(conv_5_3))

deconv_5_3 = deconv_layer(unpool_5, [3, 3, 512, 512], 512, 'deconv_5_3')
deconv_5_2 = deconv_layer(deconv_5_3, [3, 3, 512, 512], 512, 'deconv_5_2')
deconv_5_1 = deconv_layer(deconv_5_2, [3, 3, 512, 512], 512, 'deconv_5_1')

unpool_4 = unpool_layer2x2(deconv_5_1, pool_4_argmax, tf.shape(conv_4_3))

deconv_4_3 = deconv_layer(unpool_4, [3, 3, 512, 512], 512, 'deconv_4_3')
deconv_4_2 = deconv_layer(deconv_4_3, [3, 3, 512, 512], 512, 'deconv_4_2')
deconv_4_1 = deconv_layer(deconv_4_2, [3, 3, 256, 512], 256, 'deconv_4_1')

unpool_3 = unpool_layer2x2(deconv_4_1, pool_3_argmax, tf.shape(conv_3_3))

deconv_3_3 = deconv_layer(unpool_3, [3, 3, 256, 256], 256, 'deconv_3_3')
deconv_3_2 = deconv_layer(deconv_3_3, [3, 3, 256, 256], 256, 'deconv_3_2')
deconv_3_1 = deconv_layer(deconv_3_2, [3, 3, 128, 256], 128, 'deconv_3_1')

unpool_2 = unpool_layer2x2(deconv_3_1, pool_2_argmax, tf.shape(conv_2_2))

deconv_2_2 = deconv_layer(unpool_2, [3, 3, 128, 128], 128, 'deconv_2_2')
deconv_2_1 = deconv_layer(deconv_2_2, [3, 3, 64, 128], 64, 'deconv_2_1')

unpool_1 = unpool_layer2x2(deconv_2_1, pool_1_argmax, tf.shape(conv_1_2))

deconv_1_2 = deconv_layer(unpool_1, [3, 3, 64, 64], 64, 'deconv_1_2')
deconv_1_1 = deconv_layer(deconv_1_2, [3, 3, 32, 64], 32, 'deconv_1_1')

score_1 = deconv_layer(deconv_1_1, [1, 1, 21, 32], 21, 'score_1')

logits = tf.reshape(score_1, (-1, 21))
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf.reshape(expected, [-1]), name='x_entropy')
loss = tf.reduce_mean(cross_entropy, name='x_entropy_mean')

train_step = tf.train.AdamOptimizer(rate).minimize(loss)

prediction = tf.argmax(tf.reshape(tf.nn.softmax(logits), tf.shape(score_1)), dimension=3)
accuracy = tf.reduce_sum(tf.pow(prediction - expected, 2))

In [21]:
saver = tf.train.Saver(max_to_keep = 5, keep_checkpoint_every_n_hours =1)

In [22]:
def optimize(train_stage=1, training_steps=1000, restore_session=False, learning_rate=1e-6):
    if restore_session:
        step_start = restore_session()
    else:
        step_start = 0

    if train_stage == 1:
        trainset = open('data/stage_1_train_imgset/train.txt').readlines()
    else:
        trainset = open('data/stage_2_train_imgset/train.txt').readlines()

    for i in range(step_start, step_start+training_steps):
        # pick random line from file
        random_line = random.choice(trainset)
        image_file = random_line.split(' ')[0]
        ground_truth_file = random_line.split(' ')[1]
        image = np.float32(cv2.imread('data' + image_file))
        ground_truth = cv2.imread('data' + ground_truth_file[:-1], cv2.IMREAD_GRAYSCALE)
        # norm to 21 classes [0-20] (see paper)
        ground_truth = (ground_truth / 255) * 20
        print('run train step: '+str(i))
        start = time.time()
        train_step.run(session=session, feed_dict={x: [image], y_true: [ground_truth], rate: learning_rate})

        if i % 10000 == 0:
            print('step {} finished in {:.2f} s with loss of {:.6f}'.format(
                i, time.time() - start, loss.eval(session=session, feed_dict={x: [image], y_true: [ground_truth]})))
            saver.save(session, checkpoint_dir+'model', global_step=i)
            print('Model {} saved'.format(i))


In [23]:
optimize(training_steps=10)

run train step: 0


InvalidArgumentError: No OpKernel was registered to support Op 'MaxPoolWithArgmax' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[Node: MaxPoolWithArgmax_15 = MaxPoolWithArgmax[T=DT_FLOAT, Targmax=DT_INT64, ksize=[1, 2, 2, 1], padding="SAME", strides=[1, 2, 2, 1]](Relu_62)]]

Caused by op 'MaxPoolWithArgmax_15', defined at:
  File "/Users/yuravasiliuk/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/yuravasiliuk/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/yuravasiliuk/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/yuravasiliuk/anaconda3/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/yuravasiliuk/anaconda3/en